# Imports

In [7]:
import datetime
from collections import Counter, defaultdict
import os
import re

from docx import Document
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import pandas as pd
import PyPDF2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
import pymagnitude
wv = pymagnitude.Magnitude("/home/nlpsomnath/.local/lib/python3.5/site-packages/negative_news/resources/crawl-300d-2M.magnitude")

import spacy
nlp = spacy.load('en_core_web_sm')

from negative_news.consumer.utils import desm_pandas_wrapper

Using TensorFlow backend.


Configuration Parameters Loaded


# Utility Functions

In [76]:
NEGATIVE_SENTIMENT_THRESHOLD = -0.05
POSITIVE_SENTIMENT_THRESHOLD = 0.05

N_SENTENCE_SIMILARITY_THRESHOLD = 3

def extract_pdf_pages(fname):
    with open(fname, "rb") as f:
        pdf = PyPDF2.PdfFileReader(f)
        pdf_pages = [p.extractText() for p in pdf.pages]
    
    return pdf_pages

def remove_header_footer(page):
    #remove header
    t = page[3:]
    #remove footer
    t = t[:-71]
    return t

def clean_extracted_pdf_page(page):
    #remove multi-spaces
    t = re.sub(r'\s{2,}',' ', page.strip())
    #remove new lines
    t = re.sub(r'\n', '', t)
    #fix punctuation
    t = t.replace('™', "'")
    t = t.replace('ﬁ', '"')
    t = t.replace('ﬂ', '"')
    return t

def preprocess_table_of_contents(toc_page_text):
    toc_list = remove_header_footer(toc_page_text)
    toc_list = toc_list.replace('TABLE OF CONTENTS', '')
    toc_list = re.split(r'(?<=\d\.)([^\d]*?)(?:\.)', toc_list)
    toc_list = [clean_extracted_pdf_page(t) for t in toc_list]
    toc_list = [re.sub(r'[0-9]', '', t.replace('.', '')) for t in toc_list]
    toc_list = [s.strip() for s in toc_list if s.strip() != '']
    toc_list = [re.sub('\s{2,}', '', t) for t in toc_list]
    return toc_list

def get_table_of_contents_sections(
    page_text_list,
    toc_locator_func=lambda page: 'table of contents' in page.lower(),
    preprocess_func=lambda page: [i for i in page.split('\n')]
):
    sections = []
    
    for page_text in page_text_list:
        if toc_locator_func(page_text):
            return preprocess_func(page_text)
    return sections

def make_summary(text_to_summarize, lang='english', n_sentences=3):
    MIN_SUMMARY_WORDS = 10
    
    parser = PlaintextParser.from_string(text_to_summarize, Tokenizer(lang))
    summarizer = LuhnSummarizer()
    summary = [sent for sent in summarizer(parser.document, n_sentences) if len(sent.words) > MIN_SUMMARY_WORDS]
    
    return summary

def segment_pdf_by_sections(pdf_text, sections):
    section_text_list = []
    
    for i in range(0, len(sections)):
        target_section = sections[i]
        beg_idx_target = pdf_text.index(target_section)
        
        try:
            next_section = sections[i + 1]
            end_idx_target = pdf_text.index(next_section)
        except Exception:
            next_section = None
        
        if next_section is None:
            section_text_list.append(pdf_text[beg_idx_target:]) 
        else:
            section_text_list.append(pdf_text[beg_idx_target:end_idx_target])
            
    return section_text_list

def extract_most_common_ent(ent_list):
    try:
        return Counter(ent_list).most_common()[0][0]
    except Exception:
        return 'N/A'
    
def calculate_sentiment(sentence_list, n_sentence=-1):
    analyzer = SentimentIntensityAnalyzer()
    sentence_sentiment_list = []
    
    for sent_idx, sent in enumerate(sentence_list):
        try:
            sentence_sentiment_list.append(analyzer.polarity_scores(sent)['compound'])
        except Exception:
            print('Sentiment calculation failed at sentence index {}. Sentence is: {}'.format(sent_idx, sent))
            sentence_sentiment_list.append(0)
            raise
    
    if n_sentence == -1:
        doc_sentiment = np.mean(sentence_sentiment_list)
    elif n_sentence > 0:
        doc_sentiment = np.mean(sorted(sentence_sentiment_list)[:n_sentence])
    else:
        raise ValueError("Invalid value for 'n_sentence': {}".format(n_sentence))
        
    sentiment_category = sentiment_category_from_score(doc_sentiment)
    
    return sentence_sentiment_list, doc_sentiment, sentiment_category

def sentiment_category_from_score(score):
    if score <= -0.05:
        sentiment_category = 'Negative'
    elif score >= 0.05:
        sentiment_category = 'Positive'
    else:
        sentiment_category = 'Neutral'
    
    return sentiment_category

def normalize_term_similarity_score(unnormalized_score_dict, min_max_score_dict):
    normalized_score_dict = {}
    for term, score_tuples in unnormalized_score_dict.items():
        doc_idx, scores = zip(*score_tuples)
        _min = min_max_score_dict[term]['min']
        _max = min_max_score_dict[term]['max']
        normalized_scores = [((score - _min) / (_max - _min)) for score in scores]
        normalized_score_dict[term] = normalized_scores
    return normalized_score_dict

# Generate docx

In [4]:
def generate_word_output(fname, data):
    doc = Document()
    
    doc = build_overview_page(doc, data)
    
    doc.add_page_break()
    
    for s_data in data['section_data']:
        doc = build_section_detail_entry(doc, s_data)
        
    print('Saved Word document to the following path: {}'.format(fname))

    doc.save(fname)
    

def build_overview_page(doc, data):
    # build header
    doc.add_heading('Document Summarization Report'.upper(), level=0)
    
    doc.add_heading('File name: ')
    doc.add_paragraph(data['fname'])
    
    doc.add_heading('Generation date:')
    doc.add_paragraph(data['run_date'])
    
    #build doc level statistics
    doc.add_heading('Document-level Statistics:', level=1)
    
    table = doc.add_table(1, 2)
    table.style = 'LightList-Accent1'
    
    heading_cells = table.rows[0].cells
    heading_cells[0].text = 'Statistic'
    heading_cells[1].text = 'Value'
    
    rows = [
        ('Page Length', data['page_length'] + ' pages'),
        ('Character Length', data['char_length'] + ' characters'),
        ('Document Type', data['doc_type']),
        ('Number of Sections', data['num_sections'] + ' sections'),
        ('Average Section Sentiment', data['avg_sxn_sentiment'])
    ]
    
    for row in rows:
        cells = table.add_row().cells
        for idx, item in enumerate(row):
            cells[idx].text = str(item)
    
    #build section level statistics
    doc.add_heading('Section-level Statistics:', level=1)
    
    table = doc.add_table(1, 2)
    table.style = 'LightList-Accent1'
    
    heading_cells = table.rows[0].cells
    heading_cells[0].text = 'Statistic'
    heading_cells[1].text = 'Value'
    
    rows = [
        ('Positive sentiment sections', data['num_pos_sxn'] + ' sections'),
        ('Neutral sentiment sections', data['num_neu_sxn'] + ' sections'),
        ('Negative sentiment sections', data['num_neg_sxn'] + ' sections'),
        ('Unique entity mentions', data['num_unique_ents'] + ' mentions'),
    ]
    
    for row in rows:
        cells = table.add_row().cells
        for idx, item in enumerate(row):
            cells[idx].text = str(item)
            
    #build analysis terms used
    doc.add_heading('Analysis Terms Used:', level=1)
    doc.add_paragraph(', '.join(data['analysis_terms_list']))
    
    return doc
    
def build_section_detail_entry(doc, data):
    p = doc.add_paragraph("", style='List Number')
    p.add_run(data['sxn_name']).underline = True
    p.add_run(' ({}% of document)'.format(data['pct_doc_size']))
    
    p = doc.add_paragraph("")
    p.add_run('Sentiment:').underline = True
    p.add_run(' {}'.format(data['sentiment_category']))
    
    p = doc.add_paragraph("")
    p.add_run('Term similarity:').underline = True
    p.add_run(' {}'.format(', '.join(['{} ({})'.format(term, score) for term, score in data['term_similarity'].items()])))
    
    p = doc.add_paragraph("")
    p.add_run('Notable entities:').underline = True
    p.add_run(' {}'.format(', '.join(data['notable_ents'])))
    
    p = doc.add_paragraph("")
    p.add_run('Key excerpts:').underline = True
    for e in data['key_excerpts']:
        doc.add_paragraph(e, style = 'List Bullet')
        
    doc.add_paragraph()
    
    return doc

# Working version of 'main' method

In [96]:
analysis_terms = ['fraud', 'penalty', 'liability', 'claims', 'agreement', 'defendant', 'plaintiff']
fname_pdf = "/home/nlpsomnath/NegNews/Mueller/test/HSBC - LIBOR settlement $100MM 4.10.18.pdf"

#extract pdf pages 
raw_pdf_pages = extract_pdf_pages(fname_pdf)

#extract toc sections
toc_sections = get_table_of_contents_sections(raw_pdf_pages, preprocess_func=preprocess_table_of_contents)

#clean pdf pages
clean_pdf_pages = [clean_extracted_pdf_page(remove_header_footer(p)) for p in raw_pdf_pages]

#segment pdf pages by section
clean_pdf_sections = segment_pdf_by_sections(' '.join(clean_pdf_pages[2:]), toc_sections)

#create df
df = pd.DataFrame({'section_names': toc_sections, 'clean_pdf_text': clean_pdf_sections, 'clean_pdf_text_len': [len(t) for t in clean_pdf_sections]})

#sumy
df['summary'] =  df.apply(lambda row: make_summary(row['clean_pdf_text']), axis=1)

#tokenize
df['sentence_tokenized'] = df.apply(lambda row: sent_tokenize(row['clean_pdf_text']), axis=1)
df['sentence_tokenized_word_tokenized'] = df.apply(lambda row: [word_tokenize(sent) for sent in row['sentence_tokenized']], axis=1)

#tfidf
pipeline = Pipeline([
    ('tf-idf', TfidfVectorizer(sublinear_tf=True, min_df=1, norm='l2', stop_words='english', analyzer=lambda x: x))
])

vectorizer = pipeline.fit(list(df['clean_pdf_text'])).steps[0][1]
vectors = pipeline.transform(list(df['clean_pdf_text']))

model = KMeans(n_clusters=len(toc_sections), init='k-means++', max_iter=100, n_init=1)
model.fit(vectors)

#most mentioned entities
df['tagged_text'] = df.apply(lambda row: nlp(row['clean_pdf_text']), axis=1)
allowed_ents = ['ORG', 'PERSON', 'GPE', 'LAW', 'WORK_OF_ART']
df['ents'] = df.apply(lambda row: [ent.text for ent in row['tagged_text'].ents if ent.label_ in allowed_ents], axis=1)
df['top_ent'] = df.apply(lambda row: extract_most_common_ent(row['ents']), axis=1)
all_ents = [ent for ent_list in list(df['ents']) for ent in ent_list]
most_common_entities = Counter(all_ents).most_common()

#sentiment
df['sentence_sentiment'], df['section_sentiment'], df['sentiment_category'] = zip(
    *df.apply(
        lambda row: calculate_sentiment(row['sentence_tokenized'], n_sentence=5), 
        axis=1
    )
)

#term similarity
df['max_desm_score'], df['desm_sentences'], df['desm_term'], df['desm_score_dict'] = zip(
    *df.apply(
        lambda row: desm_pandas_wrapper(
            wv,
            analysis_terms,
            row
        ),
        axis = 1
    )
)

#aggregate all term similarity scores into dictionary
agg_score_dict = defaultdict(list)
for _, row in df.iterrows():
    for term, score_tuples in row['desm_score_dict'].items():
        doc_idx, scores = zip(*score_tuples)
        agg_score_dict[term].extend(scores)

#put min and max term similarity scores into dictionary
min_max_dict = defaultdict(dict)
for term in score_list.keys():
    min_max_dict[term]['min'] = min(score_list[term])
    min_max_dict[term]['max'] = max(score_list[term])
    
#normalize term similarity scores between 0 and 1
df['normalized_desm_scores'] = df.apply(lambda row: normalize_term_similarity_score(row['desm_score_dict'], min_max_dict), axis=1)

#generate output argument dictionary
fname_output = os.path.basename(fname_pdf).split('.')[0] + '.docx'
avg_sxn_sentiment = sentiment_category_from_score(np.mean(df['section_sentiment']))

section_data = []

for idx, row in df.iterrows():
    scores = []
    section_data.append(
        {
            'sxn_name': row['section_names'],
            'pct_doc_size': str(round((row['clean_pdf_text_len'] / sum(df['clean_pdf_text_len']) * 100), 2)),
            'term_similarity': {term: str(round(np.mean(scores[:N_SENTENCE_SIMILARITY_THRESHOLD]) * 100, 1) ) for term, scores in row['normalized_desm_scores'].items()},
            'sentiment_category': row['sentiment_category'],
            'notable_ents': [row['top_ent']],
            'key_excerpts': [str(summary) for summary in row['summary']],
        }
    )

data = {
    'fname': os.path.basename(fname_pdf),
    'run_date': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'page_length': str(len(raw_pdf_pages)),
    'char_length': str(sum(df['clean_pdf_text_len'])),
    'num_sections': str(len(df)),
    'avg_sxn_sentiment': avg_sxn_sentiment,
    'doc_type': 'Unknown',
    'num_pos_sxn': str(len(df[df['sentiment_category'] == 'Positive'])),
    'num_neu_sxn': str(len(df[df['sentiment_category'] == 'Neutral'])),
    'num_neg_sxn': str(len(df[df['sentiment_category'] == 'Negative'])),
    'num_unique_ents': str(len(set(all_ents))),
    'analysis_terms_list': analysis_terms,
    'section_data': section_data,
}

generate_word_output(fname_output, data)